<a href="https://colab.research.google.com/github/Tommy-Las/WatfordFC/blob/main/Data_cleaning_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# If google colab

In [233]:
# Mount drive

# from google.colab import drive
# drive.mount('/content/drive')

# Import packages and data

In [234]:
import pandas as pd
# Turn off warnings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

Import GPS data

In [235]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/GPS 2018-2023.xlsx'
file_path = 'data/GPS 2018-2023_NoContact.xlsx'
df_gps = pd.read_excel(file_path)

Import speed data

In [236]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/max_speed.xlsx'
file_path = 'data/max_speed.xlsx'
df_speed = pd.read_excel(file_path)

In [237]:

file_path = 'data/processed_newdata.xlsx'
df_new = pd.read_excel(file_path)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3251 entries, 0 to 3250
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   PlayerID     3251 non-null   int64         
 1   Injury       3251 non-null   int64         
 2   Session      3218 non-null   object        
 3   Date         3251 non-null   datetime64[ns]
 4   Mins         3251 non-null   float64       
 5   TD           3251 non-null   float64       
 6   >19.8        3251 non-null   float64       
 7   >25          3251 non-null   float64       
 8   >19.8_Rel-1  3076 non-null   float64       
 9   >25_Rel-1    3076 non-null   float64       
 10  Sprints      3251 non-null   float64       
 11  % Max Speed  3240 non-null   float64       
 12  ACC          3251 non-null   float64       
 13  DEC          3251 non-null   float64       
dtypes: datetime64[ns](1), float64(10), int64(2), object(1)
memory usage: 355.7+ KB


Import wellbeing data


In [238]:
#file_path = '/content/drive/MyDrive/WATFORD FC/Datos Wellbeing/wellbeing ssp.xlsx'
#df_wellbeing = pd.read_excel(file_path)

# Merge Sprint rows into a single row per session

The sprint values are separated in different rows per session ,

We want to merge in into a single row per session.

## Transform data before merging sprint rows

In [239]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123180 entries, 0 to 123179
Data columns (total 11 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   DATE                                         123180 non-null  object 
 1   ID                                           123180 non-null  int64  
 2   Player Position                              123155 non-null  object 
 3   Max Speed                                    116475 non-null  float64
 4   Sprints                                      113560 non-null  float64
 5   MINUTES                                      113654 non-null  float64
 6   Season                                       123180 non-null  object 
 7   Max Speed Season                             123180 non-null  float64
 8   Avg Speed Season                             122357 non-null  float64
 9   % Max Speed                                  122357 non-nul

We want to remove rows after 08-07-2021 since we don't have data for sprints and speed

In [240]:
from datetime import date
# Convert DATE column values into Pandas datetime object
df_speed['DATE'] = pd.to_datetime(df_speed['DATE'], dayfirst=True)

# Filter rows after date: '2021-08-07'
date_filter = pd.Timestamp('2021-08-07')
df_speed = df_speed[df_speed['DATE'] > date_filter]

# Filter rows after date: '2023-05-11'
date_filter = pd.Timestamp('2023-05-11')
df_speed = df_speed[df_speed['DATE'] < date_filter]

date_filter = pd.Timestamp('2021-06-28')
df_gps = df_gps[df_gps['DATE'] > date_filter]

In [241]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55362 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


Replace NULL sprint values to 0

In [242]:
# Replace NULL values from Sprint to 0
df_speed['Sprints'] = df_speed['Sprints'].fillna(0)

## Merge sprint values

Print values to compare merge later

In [243]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
58257,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
58258,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
58259,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
58260,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
58300,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
58289,2023-05-08,48692,CENTRE BACK,32.56,35.0,69.15,2022-2023,36.21,21.029270,89.919912,54.831811
58290,2023-05-08,87583,CENTRE MIDFIELDER,22.69,11.0,34.13,2022-2023,34.27,21.189939,66.209513,7.079117
58291,2023-05-08,37537,CENTRE MIDFIELDER,28.65,5.0,47.05,2022-2023,34.09,21.974456,84.042241,30.378655
58292,2023-05-08,37537,CENTRE MIDFIELDER,30.59,3.0,22.10,2022-2023,34.09,21.974456,89.733060,39.207088
58293,2023-05-08,37537,CENTRE MIDFIELDER,30.59,8.0,69.15,2022-2023,34.09,21.974456,89.733060,39.207088


Do a group by player and date, and select the maximum value for each feature

In [244]:
df_speed = df_speed.groupby(['DATE', 'ID']).agg('max').reset_index()

In [245]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
8262,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
8261,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
8260,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
8259,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
8249,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
8240,2023-05-08,10103,FULL BACK,21.24,1.0,44.84,2022-2023,34.95,22.202230,60.772532,-4.333932
8241,2023-05-08,12086,WINGER,35.04,41.0,88.27,2022-2023,35.44,21.901184,98.871332,59.991349
8242,2023-05-08,19817,FULL BACK,31.68,0.0,128.04,2022-2023,35.77,21.605460,88.565837,46.629599
8243,2023-05-08,21079,WINGER,34.51,51.0,88.04,2022-2023,36.97,22.708654,93.345956,51.968499
8244,2023-05-08,23085,CENTRE MIDFIELDER,23.71,7.0,58.52,2022-2023,32.12,20.304388,73.816936,16.772788


# Handle duplicates for GPS Data

There are some players that have 2 records for the same day, they are different amounts so we sum them

In [246]:
# Define columns to sum
columns_to_sum = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC']

# Define columns to select the first value
columns_to_first = ['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE', 'preseason-season', 'MANAGER']

# Group by the duplicate subset and aggregate
df_gps_aggregated = (
    df_gps[df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)]
    .groupby(['PLAYER', 'DATE'], as_index=False)
    .agg({**{col: 'sum' for col in columns_to_sum},
          **{col: 'first' for col in columns_to_first}})
)

# Ensure non-duplicated rows are preserved by combining them back
df_gps_combined = pd.concat([
    df_gps[~df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)],
    df_gps_aggregated
], ignore_index=True)

# Merge GPS and Speed dataframes

## Prepare the data before the merge

Verify column names for both Dataframes

In [247]:
df_gps_combined.columns

Index(['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE',
       'preseason-season', 'MANAGER', 'Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC'],
      dtype='object')

In [248]:
df_speed.columns

Index(['DATE', 'ID', 'Player Position', 'Max Speed', 'Sprints', 'MINUTES',
       'Season', 'Max Speed Season', 'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average'],
      dtype='object')

Change datatypes before the merge

In [249]:
# Convert DATE column from GPS df into datetime pandas obkect
df_gps_combined['DATE'] = pd.to_datetime(df_gps_combined['DATE'], dayfirst=True)

# Drop NULL values for 'PLAYER'
df_gps_combined = df_gps_combined.dropna(subset=['PLAYER'])

# Convert ID and PLAYER columns to the same data type - integers
df_gps_combined['PLAYER'] = df_gps_combined['PLAYER'].astype(int)
df_speed['ID'] = df_speed['ID'].astype(int)

## Do the merge of gps & speed dataframes

In [250]:
# Perform an inner join on matching DATE and PLAYER/ID values
df_merged = df_gps_combined.merge(df_speed, left_on=['DATE', 'PLAYER'], right_on=['DATE', 'ID'], how='inner')

In [251]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7831 entries, 0 to 7830
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7831 non-null   datetime64[ns]
 1   Column2                                      7831 non-null   object        
 2   PLAYER                                       7831 non-null   int64         
 3   Injury                                       7831 non-null   float64       
 4   season                                       7831 non-null   object        
 5   LEAGUE                                       7831 non-null   object        
 6   preseason-season                             7831 non-null   object        
 7   MANAGER                                      7831 non-null   object        
 8   Total D                                      7831 non-null   object        
 9

Verify class imbalances

In [252]:
print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

No Injury: 7794
Injury: 37
%: 0.47%


# Merge df with 'weight' in wellbeing dataframe

In [253]:
# df_wellbeing = df_wellbeing[["Weight", "Date", "PLAYER"]]
# df_wellbeing.info()

In [254]:
# # Sort by PLAYER and DATE to ensure correct order for backfilling
# df_wellbeing = df_wellbeing.sort_values(by=['PLAYER', 'Date'])

# # Fill the NULL values in 'Weight' with the previous valid value for each player
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].bfill()
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].ffill()

# df_wellbeing.info()

In [255]:
# # Merge the DataFrames
# df_merged = df_merged.merge(df_wellbeing, left_on=['DATE', 'PLAYER'], right_on=['Date', 'PLAYER'], how='left')

# # Sort by PLAYER and DATE to ensure chronological order
# df_merged = df_merged.sort_values(by=['PLAYER', 'DATE'])

# # Back-fill null values in 'Weight' for each PLAYER
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].bfill()
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].ffill()

# # Optional: Drop the 'Date' column from the right DataFrame if not needed
# df_merged = df_merged.drop(columns=['Date'])

# # Display the final DataFrame
# df_merged.info()

In [256]:
#df_merged.groupby('PLAYER')['Weight'].apply(lambda x: x.isnull().sum())

# Transform numeric columns to the same data types

In [257]:
cols = ['Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC', 'ID', 'Max Speed', 'Sprints', 'MINUTES', 'Max Speed Season',
       'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average']

df_merged[cols] = df_merged[cols].astype(float)

In [258]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7831 entries, 0 to 7830
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7831 non-null   datetime64[ns]
 1   Column2                                      7831 non-null   object        
 2   PLAYER                                       7831 non-null   int64         
 3   Injury                                       7831 non-null   float64       
 4   season                                       7831 non-null   object        
 5   LEAGUE                                       7831 non-null   object        
 6   preseason-season                             7831 non-null   object        
 7   MANAGER                                      7831 non-null   object        
 8   Total D                                      7831 non-null   float64       
 9

# Values that are 0 for ACC, DEC, Max Speed

Perform imputations on missing data

In [259]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7794
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,3
9,>19.8,420


Perform KNN Imputation

In [260]:
# Columns to impute
metrics_imputation_zeroes = ['Total D', 'ACC', 'DEC', 'Max Speed', 'MINUTES']

# Columns to use for KNN calculation
columns_for_knn = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC', 'ID',
                   'Max Speed', 'Sprints', 'MINUTES', '% Max Speed']

# Replace 0 with NaN in the columns to be imputed
df_merged[metrics_imputation_zeroes] = df_merged[metrics_imputation_zeroes].replace(0, np.nan)

# Ensure the columns used for KNN calculation are numeric
columns_for_knn_numeric = df_merged[columns_for_knn].select_dtypes(include=[np.number]).columns.tolist()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # Adjust n_neighbors if needed

# Perform KNN imputation using the broader set of columns for calculation
imputed_values = knn_imputer.fit_transform(df_merged[columns_for_knn_numeric])

# Update only the specified columns to impute
df_merged[metrics_imputation_zeroes] = imputed_values[:,
    [columns_for_knn_numeric.index(col) for col in metrics_imputation_zeroes]]

In [261]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7794
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,0
9,>19.8,420


Calculate '% Max Speed' and '% Speed difference against max. Speed average'

In [262]:
# Calculate '% Max Speed' for all rows
df_merged['% Max Speed'] = (df_merged['Max Speed'] / df_merged['Max Speed Season']) * 100

# Calculate '% Speed difference against max. Speed average' for all rows
df_merged['%Speed diference against max. Speed average'] = (
    (df_merged['Max Speed'] - df_merged['Avg Speed Season']) / df_merged['Avg Speed Season']
) * 100

# Reduce data volume/size

## Players who have never been injured

We are going to filter by players that had at least one injury, to reduce # of rows and hopefully improve accuracy

In [263]:
# Group by ID and sum the Injury column
injury_counts = df_merged.groupby('ID').agg({'Injury': 'sum'})

# Get the IDs where the sum of Injury is greater than 0
ids_to_remove = injury_counts[injury_counts['Injury'] == 0].index.tolist()

# Remove these IDs from the original DataFrame
df_filtered = df_merged[~df_merged['ID'].isin(ids_to_remove)]

In [264]:
print("No Injury: " + str(df_filtered[df_filtered["Injury"] == 0].shape[0]))
print("Injury: " + str(df_filtered[df_filtered["Injury"] == 1].shape[0]))
print(f"%: {((df_filtered[df_filtered['Injury'] == 1].shape[0] / df_filtered.shape[0]) * 100):.2f}%")

No Injury: 4683
Injury: 37
%: 0.78%


Verify that at least one player had an injury

In [265]:
df_filtered.groupby('ID').agg({'Injury': 'sum'})

,Injury
ID,
10103.0,4.0
10452.0,2.0
12086.0,1.0
17316.0,2.0
18096.0,3.0
20083.0,1.0
21079.0,2.0
23081.0,2.0
23085.0,2.0


# Change column names & drop unnecesary columns

In [266]:
column_rename_dict = {
    'Column2': 'Session',
    'DATE': 'DATE',
    'ID': 'PlayerID',
    'Total D': 'TD',
    '>19.8': '>19.8',
    '> 25 Km/h': '>25',
    'ACC': 'ACC',
    'DEC': 'DEC',
    'Max Speed': 'Max Speed',
    'Max Speed Season': 'Max Speed Season',
    'Avg Speed Season': 'Avg Speed Season',
    '% Max Speed': '% Max Speed',
    '%Speed diference against max. Speed average': 'Speed Diff Max Avg',
    'Injury': 'Injury',
    'MINUTES': 'Mins',
    'Sprints': 'Sprints',
    'Total D_Rel': 'TD_Rel',
    '>19.8_Rel': '>19.8_Rel',
    '> 25 Km/h_Rel': '>25 Km/h_Rel',
    'ACC_Rel': 'ACC_Rel',
    'DEC_Rel': 'DEC_Rel',
    'Sprints_Rel': 'Sprints_Rel'
}

df_filtered.rename(columns=column_rename_dict, inplace=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4720 entries, 3 to 7798
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4720 non-null   datetime64[ns]
 1   Session             4720 non-null   object        
 2   PLAYER              4720 non-null   int64         
 3   Injury              4720 non-null   float64       
 4   season              4720 non-null   object        
 5   LEAGUE              4720 non-null   object        
 6   preseason-season    4720 non-null   object        
 7   MANAGER             4720 non-null   object        
 8   TD                  4720 non-null   float64       
 9   >19.8               4720 non-null   float64       
 10  >25                 4720 non-null   float64       
 11  ACC                 4720 non-null   float64       
 12  DEC                 4720 non-null   float64       
 13  PlayerID            4720 non-null   float64       
 1

In [267]:
df_filtered = df_filtered.drop(columns=['LEAGUE', 'MANAGER', 'PLAYER', 'Season'])

# Calculate relative values

Remove spaces in 'Microcycle' column

In [268]:
df_filtered['Session'] = df_filtered['Session'].str.replace(' ', '', regex=False)
df_filtered["Session"].unique()

array(['MD', 'M-1', 'M-2', 'M-3', 'M-5', 'M+3', 'M+2', 'M-4', 'M+1',
       'MD(AWAY)', 'MD(HOME)', nan], dtype=object)

To calculate the relative values, we want the max and avg values for players that played over 85 in a Match Day

In [269]:
def calculate_relative_values(df_original, metrics):
    # Array that holds match days
    session_values = ['MD', 'MD(HOME)', 'MD(AWAY)']

    # Do a copy of the original DF
    df_copy = df_original.copy()

    # Filter players with +85 mins and match day
    df_filtered_85_md = df_copy[
        (df_copy['Mins'] > 85) &
        (df_copy['Session'].isin(session_values))
    ]

    # Players who never completed 85 mins in matchday
    df_filtered_85_only_mins = df_copy[df_copy['Mins'] > 85]

    # Combine both DataFrames
    df_filtered_85 = pd.concat([df_filtered_85_md, df_filtered_85_only_mins])

    # Temporary dictionaries to hold max and avg values per player
    player_max = {}
    player_avg = {}

    # Calculate max and avg for each player
    for player in df_filtered_85['PlayerID'].unique():
        player_data = df_filtered_85[df_filtered_85['PlayerID'] == player]
        player_max[player] = player_data[metrics].max()
        player_avg[player] = player_data[metrics].mean()

    # Add relative values to the original DataFrame
    for metric in metrics:

        df_original[f"{metric}_Rel"] = np.nan

        for player in df_original['PlayerID'].unique():
            if player in player_max and player in player_avg:
                max_value = player_max[player][metric]
                avg_value = player_avg[player][metric]

                # Compute relative value
                df_original.loc[df_original['PlayerID'] == player, f"{metric}_Rel"] = (
                    (df_original.loc[df_original['PlayerID'] == player, metric] * 100) /
                    ((max_value + avg_value) / 2)
                ).round(2)

    return df_original

Call function to calculate relative values

In [270]:
metrics_rel = ['>19.8', '>25']

# Avoid for now
df_rel = calculate_relative_values(df_filtered, metrics_rel)

# df_rel = df_filtered.copy()
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4720 entries, 3 to 7798
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4720 non-null   datetime64[ns]
 1   Session             4505 non-null   object        
 2   Injury              4720 non-null   float64       
 3   season              4720 non-null   object        
 4   preseason-season    4720 non-null   object        
 5   TD                  4720 non-null   float64       
 6   >19.8               4720 non-null   float64       
 7   >25                 4720 non-null   float64       
 8   ACC                 4720 non-null   float64       
 9   DEC                 4720 non-null   float64       
 10  PlayerID            4720 non-null   float64       
 11  Player Position     4720 non-null   object        
 12  Max Speed           4720 non-null   float64       
 13  Sprints             4720 non-null   float64       
 1

Merge with new dataset

In [271]:
# Rename columns in df_new
df_new["Date"] = pd.to_datetime(df_new["Date"], dayfirst=True)
df_new.rename(columns={'Date': 'DATE', '>19.8_Rel-1': '>19.8_Rel', '>25_Rel-1': '>25_Rel'}, inplace=True)
df_new = df_new[df_new["Session"].isin(['MD', 'MD-1', 'MD-2', 'MD-3', 'MD-5', 'MD+3', 'MD+2', 'MD-4', 'MD+1'])]

# Concatenate the dataframes
df_rel_concatenated = pd.concat([df_rel, df_new], axis=0)
df_rel_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7641 entries, 3 to 3250
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7641 non-null   datetime64[ns]
 1   Session             7426 non-null   object        
 2   Injury              7641 non-null   float64       
 3   season              4720 non-null   object        
 4   preseason-season    4720 non-null   object        
 5   TD                  7641 non-null   float64       
 6   >19.8               7641 non-null   float64       
 7   >25                 7641 non-null   float64       
 8   ACC                 7641 non-null   float64       
 9   DEC                 7641 non-null   float64       
 10  PlayerID            7641 non-null   float64       
 11  Player Position     4720 non-null   object        
 12  Max Speed           4720 non-null   float64       
 13  Sprints             7641 non-null   float64       
 1

In [272]:
md_values = ['MD', 'MD(HOME)', 'MD(AWAY)']
df_rel_concatenated['Session'] = df_rel_concatenated['Session'].replace(md_values, 'MD')

In [273]:
# Dictionary to map old session values to new ones
session_mapping = {
    'M-1': 'MD-1', 'M-2': 'MD-2', 'M-3': 'MD-3', 'M-5': 'MD-5',
    'M+3': 'MD+3', 'M+2': 'MD+2', 'M-4': 'MD-4', 'M+1': 'MD+1'
}

# Replace the specified values with the new 'MD' values
df_rel_concatenated['Session'] = df_rel_concatenated['Session'].replace(session_mapping)

# Verify the unique values in the 'Session' column
df_rel_concatenated['Session'].unique()

array(['MD', 'MD-1', 'MD-2', 'MD-3', 'MD-5', 'MD+3', 'MD+2', 'MD-4',
       'MD+1', nan], dtype=object)

# Function for densities

In [42]:
# cols_mins = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints']

# # Create new columns by dividing by the 'Minutes' column
# for col in cols_mins:
#     df_rel[f'{col}/Mins'] = df_rel[col] / df_rel['Mins']

In [43]:
# df_rel['Sprints/Mins'] = df_rel['Sprints/Mins'].fillna(0)

# Player loads

## Calculate loads

In [274]:
def calcular_acumulado(df, columnas_calcular, dias):
    # Columns to exclude when the day is 3
    excluded_columns_3_days = ['>19.8_Rel', '>25_Rel', '% Max Speed', 'Mins']

    # Create an empty list to store processed player DataFrames
    processed_players = []

    # Process each player separately
    for player_id in df['PlayerID'].unique():
        # Filter data for the current player
        player_data = df[df['PlayerID'] == player_id].copy()

        # Create a full date range for the player (from the first to the last recorded date)
        full_date_range = pd.date_range(start=player_data['DATE'].min(), end=player_data['DATE'].max(), freq='D')

        # Set 'DATE' as the index and reindex to fill missing dates with zeros
        player_data = player_data.set_index('DATE').reindex(full_date_range, fill_value=0).reset_index()
        player_data.rename(columns={'index': 'DATE'}, inplace=True)
        player_data['PlayerID'] = player_id

        # Perform rolling calculations for each metric, excluding the current day
        for dia in dias:
            for col in columnas_calcular:
                # Skip excluded columns when the day is 3
                if dia != 1 and col in excluded_columns_3_days:
                    continue

                # Check if the column exists to avoid errors
                if col in player_data.columns:
                    if dia in [7, 21]:
                        # Rolling sum, mean, and std for 7 and 21 days
                        player_data[f'{col}-{dia}'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                        )
                        player_data[f'{col}-{dia}-avg'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).mean()
                        )
                        player_data[f'{col}-{dia}-std'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).std()
                        )
                    else:
                        # Only rolling sum for other periods
                        player_data[f'{col}-{dia}'] = (
                            player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                        )

        # Drop rows where all calculated values are zero (rest days)
        mask_non_zero = (player_data[columnas_calcular].sum(axis=1) > 0)
        player_data = player_data[mask_non_zero]

        # Append the processed player's data to the list
        processed_players.append(player_data)

    # Concatenate all processed player DataFrames into a single DataFrame
    df_resultado = pd.concat(processed_players, ignore_index=True)

    return df_resultado


In [316]:
cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC', 'Sprints', 'Mins', '>19.8_Rel', '>25_Rel', '% Max Speed']

cumulative_df = calcular_acumulado(df_rel_concatenated, cols_calculate, [1,3,7,21]) # loads -1, -3, -7, -21

In [317]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7641 entries, 0 to 7640
Data columns (total 73 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                7641 non-null   datetime64[ns]
 1   Session             7426 non-null   object        
 2   Injury              7641 non-null   float64       
 3   season              4720 non-null   object        
 4   preseason-season    4720 non-null   object        
 5   TD                  7641 non-null   float64       
 6   >19.8               7641 non-null   float64       
 7   >25                 7641 non-null   float64       
 8   ACC                 7641 non-null   float64       
 9   DEC                 7641 non-null   float64       
 10  PlayerID            7641 non-null   float64       
 11  Player Position     4720 non-null   object        
 12  Max Speed           4720 non-null   float64       
 13  Sprints             7641 non-null   float64     

In [318]:
cumulative_df.sort_values(by="DATE", ascending=False).head()

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,TD-7-avg,TD-7-std,>19.8-7,>19.8-7-avg,>19.8-7-std,>25-7,>25-7-avg,>25-7-std,ACC-7,ACC-7-avg,ACC-7-std,DEC-7,DEC-7-avg,DEC-7-std,Sprints-7,Sprints-7-avg,Sprints-7-std,TD-21,TD-21-avg,TD-21-std,>19.8-21,>19.8-21-avg,>19.8-21-std,>25-21,>25-21-avg,>25-21-std,ACC-21,ACC-21-avg,ACC-21-std,DEC-21,DEC-21-avg,DEC-21-std,Sprints-21,Sprints-21-avg,Sprints-21-std
7640,2025-02-01,MD,0.0,NaN,NaN,10383.19,327.09,133.93,121.0,135.0,177.0,NaN,NaN,9.0,116.54,NaN,NaN,94.92,NaN,0.00,0.00,2673.04,30.45,0.00,53.0,48.0,0.0,62.0,0.00,0.0,70.02,8484.97,359.24,166.74,113.0,113.0,8.0,17505.93,2500.847143,2057.319044,667.43,95.347143,122.508898,212.16,30.308571,62.125843,219.0,31.285714,30.641553,205.0,29.285714,27.505844,11.0,1.571429,3.047247,55719.54,3277.620000,2743.546004,2123.61,124.918235,140.945838,494.05,29.061765,45.230420,785.0,46.176471,37.954307,760.0,44.705882,39.504374,33.0,1.941176,2.585309
5695,2025-02-01,MD,0.0,NaN,NaN,10981.08,516.02,64.50,132.0,138.0,45.0,NaN,NaN,7.0,98.92,NaN,NaN,84.14,NaN,39.33,0.00,2076.10,6.08,0.00,32.0,25.0,0.0,31.0,0.00,0.0,63.16,8865.06,436.77,88.28,99.0,90.0,6.0,24785.39,3540.770000,3733.293339,962.68,137.525714,176.799624,128.46,18.351429,28.610946,278.0,39.714286,42.097958,291.0,41.571429,50.228999,10.0,1.428571,1.902379,78714.03,3748.287143,3463.152493,2859.37,136.160476,182.780364,599.58,28.551429,48.409010,907.0,43.190476,40.460622,899.0,42.809524,44.196854,37.0,1.761905,2.844376
7615,2025-02-01,MD,0.0,NaN,NaN,3082.34,336.42,84.81,23.0,10.0,104.0,NaN,NaN,2.0,40.16,NaN,NaN,82.17,NaN,33.02,0.00,2086.96,2.94,0.00,43.0,39.0,0.0,31.0,0.00,0.0,61.48,6278.08,334.03,105.08,87.0,84.0,7.0,10941.40,1563.057143,2104.178735,417.15,59.592857,123.611627,105.08,15.011429,39.716507,148.0,21.142857,27.008817,117.0,16.714286,21.132237,7.0,1.000000,2.645751,36876.34,1756.016190,1892.431056,1932.20,92.009524,140.342162,377.11,17.957619,34.389001,493.0,23.476190,28.671622,379.0,18.047619,24.003075,22.0,1.047619,2.060975
4845,2025-02-01,MD,0.0,NaN,NaN,9682.92,678.90,216.61,106.0,103.0,136.0,NaN,NaN,13.0,98.92,NaN,NaN,87.80,NaN,55.15,24.47,2362.12,127.67,20.86,44.0,34.0,2.0,62.0,8.71,0.0,84.47,8611.60,494.34,334.16,101.0,93.0,14.0,19417.92,2773.988571,2325.525781,823.84,117.691429,96.499676,507.60,72.514286,108.216720,231.0,33.000000,30.697448,209.0,29.857143,26.124245,22.0,3.142857,3.891382,56489.87,2689.993810,1715.913934,2221.65,105.792857,125.109528,1060.78,50.513333,81.778599,757.0,36.047619,27.043439,638.0,30.380952,23.414688,55.0,2.619048,3.293572
7582,2025-02-01,MD,0.0,NaN,NaN,1241.67,367.42,8.13,18.0,9.0,4.0,NaN,NaN,1.0,8.62,NaN,NaN,79.42,NaN,0.00,0.00,2660.85,17.99,0.00,53.0,47.0,0.0,62.0,0.00,0.0,66.61,6943.57,314.69,72.40,108.0,97.0,3.0,15807.07,2258.152857,2237.615587,838.35,119.764286,163.487468,134.48,19.211429,30.057604,238.0,34.000000,35.707142,202.0,28.857143,31.371051,6.0,0.857143,1.214986,72971.70,3474.842857,3850.598406,4040.53,192.406190,240.981871,794.03,37.810952,66.695186,870.0,41.428571,43.215242,720.0,34.285714,37.797014,45.0,2.142857,4.629100


## For player loads, drop rows with NULL or 0 values

In [319]:
cumulative_df.columns

Index(['DATE', 'Session', 'Injury', 'season', 'preseason-season', 'TD',
       '>19.8', '>25', 'ACC', 'DEC', 'PlayerID', 'Player Position',
       'Max Speed', 'Sprints', 'Mins', 'Max Speed Season', 'Avg Speed Season',
       '% Max Speed', 'Speed Diff Max Avg', '>19.8_Rel', '>25_Rel', 'TD-1',
       '>19.8-1', '>25-1', 'ACC-1', 'DEC-1', 'Sprints-1', 'Mins-1',
       '>19.8_Rel-1', '>25_Rel-1', '% Max Speed-1', 'TD-3', '>19.8-3', '>25-3',
       'ACC-3', 'DEC-3', 'Sprints-3', 'TD-7', 'TD-7-avg', 'TD-7-std',
       '>19.8-7', '>19.8-7-avg', '>19.8-7-std', '>25-7', '>25-7-avg',
       '>25-7-std', 'ACC-7', 'ACC-7-avg', 'ACC-7-std', 'DEC-7', 'DEC-7-avg',
       'DEC-7-std', 'Sprints-7', 'Sprints-7-avg', 'Sprints-7-std', 'TD-21',
       'TD-21-avg', 'TD-21-std', '>19.8-21', '>19.8-21-avg', '>19.8-21-std',
       '>25-21', '>25-21-avg', '>25-21-std', 'ACC-21', 'ACC-21-avg',
       'ACC-21-std', 'DEC-21', 'DEC-21-avg', 'DEC-21-std', 'Sprints-21',
       'Sprints-21-avg', 'Sprints-21-std'],
 

In [320]:
cols_drop = ['Max Speed','Speed Diff Max Avg', 'Max Speed Season', 'Avg Speed Season','Player Position']
cumulative_df.drop(columns=cols_drop, inplace=True)

In [321]:
# Specify the columns to check for NULL values
columns_to_check = [
    'TD-1', '>19.8-1', '>25-1', 'ACC-1', 'DEC-1', 
    'Sprints-1', 'Mins-1', '>19.8_Rel-1', '>25_Rel-1', '% Max Speed-1', 
    'TD-3', '>19.8-3', '>25-3', 'ACC-3', 'DEC-3', 'Sprints-3', 'TD-7', 
    'TD-7-avg', 'TD-7-std', '>19.8-7', '>19.8-7-avg', '>19.8-7-std', 
    '>25-7', '>25-7-avg', '>25-7-std', 'ACC-7', 'ACC-7-avg', 'ACC-7-std', 
    'DEC-7', 'DEC-7-avg', 'DEC-7-std', 'Sprints-7', 'Sprints-7-avg', 
    'Sprints-7-std', 'TD-21', 'TD-21-avg', '>19.8-21', '>19.8-21-avg', 
    '>25-21', '>25-21-avg', 'ACC-21', 'ACC-21-avg', 'DEC-21', 'DEC-21-avg', 
    'Sprints-21', 'Sprints-21-avg', 'Sprints-21-std'
]

In [322]:
cumulative_df[cumulative_df["Injury"]==1].sort_values(by="DATE", ascending=False).head(1)

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,TD-7-avg,TD-7-std,>19.8-7,>19.8-7-avg,>19.8-7-std,>25-7,>25-7-avg,>25-7-std,ACC-7,ACC-7-avg,ACC-7-std,DEC-7,DEC-7-avg,DEC-7-std,Sprints-7,Sprints-7-avg,Sprints-7-std,TD-21,TD-21-avg,TD-21-std,>19.8-21,>19.8-21-avg,>19.8-21-std,>25-21,>25-21-avg,>25-21-std,ACC-21,ACC-21-avg,ACC-21-std,DEC-21,DEC-21-avg,DEC-21-std,Sprints-21,Sprints-21-avg,Sprints-21-std
4958,2025-01-21,MD,1.0,NaN,NaN,2609.71,93.23,5.29,29.0,27.0,98.0,1.0,54.05,80.66,NaN,NaN,2604.79,60.49,25.43,35.0,24.0,2.0,55.0,0.0,0.0,77.71,9858.1,548.54,216.9,124.0,127.0,17.0,22716.64,3245.234286,3439.149911,1098.29,156.898571,234.255343,455.52,65.074286,103.766012,300.0,42.857143,43.233585,274.0,39.142857,46.34087,27.0,3.857143,6.12178,62579.206,2979.96219,2942.497236,2763.4,131.590476,185.405278,1180.55,56.216667,89.525954,751.7,35.795238,35.211653,672.8,32.038095,38.391802,61.0,2.904762,4.459874


Drop rows that contain more than 2 NULL values

In [323]:
# # Filter for rows where Injury == 1
# injury_df = cumulative_df[cumulative_df['Injury'] == 1]

# # Find rows with more than 2 NaN values in the specified columns
# rows_with_nan = injury_df[injury_df[columns_to_check].isna().sum(axis=1) > 2]

# # Display the first few rows
# rows_with_nan.head()

# # Drop rows where 2 or more NULL values exist within the specified columns
# cumulative_df = cumulative_df[cumulative_df[columns_to_check].notna().sum(axis=1) > (len(columns_to_check) - 2)]

# # Display the resulting DataFrame
# cumulative_df.info()

# Drop rows where 2 or more NaN values exist within the specified columns
cumulative_df = cumulative_df[cumulative_df[columns_to_check].isna().sum(axis=1) < 2]

# Display the resulting DataFrame info
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7496 entries, 2 to 7640
Data columns (total 68 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              7496 non-null   datetime64[ns]
 1   Session           7281 non-null   object        
 2   Injury            7496 non-null   float64       
 3   season            4677 non-null   object        
 4   preseason-season  4677 non-null   object        
 5   TD                7496 non-null   float64       
 6   >19.8             7496 non-null   float64       
 7   >25               7496 non-null   float64       
 8   ACC               7496 non-null   float64       
 9   DEC               7496 non-null   float64       
 10  PlayerID          7496 non-null   float64       
 11  Sprints           7496 non-null   float64       
 12  Mins              7496 non-null   float64       
 13  % Max Speed       7493 non-null   float64       
 14  >19.8_Rel         7362 non-nu

In [324]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 7449
Injury: 47
%: 0.63%


In [325]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Session,0
2,Injury,7449
3,season,0
4,preseason-season,0
...,...,...
63,DEC-21-avg,65
64,DEC-21-std,67
65,Sprints-21,219
66,Sprints-21-avg,219


In [326]:
cumulative_df[cumulative_df["Injury"]==1].sort_values(by="DATE", ascending=False).head(20)

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,TD-7-avg,TD-7-std,>19.8-7,>19.8-7-avg,>19.8-7-std,>25-7,>25-7-avg,>25-7-std,ACC-7,ACC-7-avg,ACC-7-std,DEC-7,DEC-7-avg,DEC-7-std,Sprints-7,Sprints-7-avg,Sprints-7-std,TD-21,TD-21-avg,TD-21-std,>19.8-21,>19.8-21-avg,>19.8-21-std,>25-21,>25-21-avg,>25-21-std,ACC-21,ACC-21-avg,ACC-21-std,DEC-21,DEC-21-avg,DEC-21-std,Sprints-21,Sprints-21-avg,Sprints-21-std
4958,2025-01-21,MD,1.0,NaN,NaN,2609.71,93.23,5.29,29.000000,27.0,98.0,1.000000,54.05,80.660000,NaN,NaN,2604.79,60.49,25.43,35.0,24.0,2.0,55.00,0.00,0.00,77.710000,9858.10,548.54,216.90,124.0,127.0,17.0,22716.64,3245.234286,3439.149911,1098.29,156.898571,234.255343,455.52,65.074286,103.766012,300.0,42.857143,43.233585,274.0,39.142857,46.340870,27.0,3.857143,6.121780,62579.206000,2979.962190,2942.497236,2763.400000,131.590476,185.405278,1180.55,56.216667,89.525954,751.700000,35.795238,35.211653,672.800000,32.038095,38.391802,61.000000,2.904762,4.459874
5181,2025-01-16,MD-2,1.0,NaN,NaN,411.78,0.00,0.00,0.000000,0.0,53.0,0.000000,16.00,34.830000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,12430.45,414.67,83.86,152.0,138.0,7.0,25027.10,3575.300000,3362.419767,704.82,100.688571,136.256679,189.83,27.118571,31.913443,273.0,39.000000,38.669540,240.0,34.285714,35.466953,14.0,2.000000,2.516611,76136.169048,3625.531859,3896.087201,3071.535556,146.263598,210.746036,1000.68,47.651429,83.474746,805.190476,38.342404,43.270689,697.111111,33.195767,39.258040,53.285714,2.537415,4.141356
7463,2025-01-09,MD,1.0,NaN,NaN,4351.15,189.35,24.06,59.000000,47.0,134.0,2.000000,60.00,82.770000,NaN,NaN,2268.61,34.60,0.00,34.0,32.0,0.0,39.00,0.00,0.00,70.590000,10428.79,484.59,25.19,146.0,118.0,3.0,16417.01,2345.287143,1970.188352,892.67,127.524286,181.323529,62.80,8.971429,15.735536,211.0,30.142857,25.504435,168.0,24.000000,23.057898,5.0,0.714286,1.253566,57490.500000,2737.642857,2435.139516,1899.410000,90.448095,144.694518,178.88,8.518095,19.061354,735.000000,35.000000,38.158878,607.000000,28.904762,39.007569,13.000000,0.619048,1.499206
6414,2024-12-23,MD-3,1.0,NaN,NaN,3994.16,462.50,33.30,74.000000,55.0,105.0,1.000000,60.00,92.570000,5.330000,9.860000,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,2816.16,138.33,26.75,32.0,25.0,3.0,15211.12,2173.017143,2450.622356,597.51,85.358571,117.397147,158.93,22.704286,35.766587,194.0,27.714286,33.609097,179.0,25.571429,33.195668,11.0,1.571429,2.299068,41009.890000,1952.851905,2206.297956,2000.440000,95.259048,126.222442,857.52,40.834286,79.530525,460.000000,21.904762,26.868020,470.000000,22.380952,26.971237,44.000000,2.095238,3.238900
6955,2024-11-14,MD-1,1.0,NaN,NaN,2426.09,13.75,0.00,43.000000,23.0,36.0,0.000000,44.00,61.930000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,6139.73,877.104286,1510.939713,154.05,22.007143,45.433896,90.58,12.940000,30.440757,74.0,10.571429,22.411732,48.0,6.857143,14.915636,3.0,0.428571,0.786796,56441.130000,2687.672857,2291.591531,2358.550000,112.311905,141.701254,640.10,30.480952,48.051034,789.000000,37.571429,37.917768,539.000000,25.666667,26.663333,35.000000,1.666667,2.496664
7495,2024-10-19,MD,1.0,NaN,NaN,7690.38,280.94,56.40,79.000000,75.0,16.0,7.000000,96.04,96.130000,106.820000,6.600000,3499.06,55.40,118.60,45.0,32.0,2.0,70.00,32.90,98.60,96.810000,5267.59,55.40,118.60,56.0,39.0,2.0,5267.59,752.512857,1378.839490,55.40,7.914286,20.939232,118.60,16.942857,44.826586,56.0,8.000000,16.822604,39.0,5.571429,11.942322,2.0,0.285714,0.755929,38374.130000,1827.339524,3183.132602,903.800000,43.038095,96.199996,387.97,18.474762,39.770093,366.000000,17.428571,29.736462,350.000000,16.666667,30.643651,19.000000,0.904762,2.047065
7492,2024-10-05,MD,1.0,NaN,NaN,9989.17,275.93,110.63,93.000000,98.0,16.0,6.000000,122.26,96.73

Remove rows where 3 or more rows contain 0 for the loads

In [184]:
cumulative_df.columns

Index(['DATE', 'Session', 'Injury', 'season', 'preseason-season', 'TD',
       '>19.8', '>25', 'ACC', 'DEC', 'PlayerID', 'Sprints', 'Mins',
       '% Max Speed', '>19.8_Rel', '>25_Rel', 'TD-1', '>19.8-1', '>25-1',
       'ACC-1', 'DEC-1', 'Sprints-1', 'Mins-1', '>19.8_Rel-1', '>25_Rel-1',
       '% Max Speed-1', 'TD-3', '>19.8-3', '>25-3', 'ACC-3', 'DEC-3',
       'Sprints-3', 'TD-7', 'TD-7-avg', 'TD-7-std', '>19.8-7', '>19.8-7-avg',
       '>19.8-7-std', '>25-7', '>25-7-avg', '>25-7-std', 'ACC-7', 'ACC-7-avg',
       'ACC-7-std', 'DEC-7', 'DEC-7-avg', 'DEC-7-std', 'Sprints-7',
       'Sprints-7-avg', 'Sprints-7-std', 'TD-21', 'TD-21-avg', 'TD-21-std',
       '>19.8-21', '>19.8-21-avg', '>19.8-21-std', '>25-21', '>25-21-avg',
       '>25-21-std', 'ACC-21', 'ACC-21-avg', 'ACC-21-std', 'DEC-21',
       'DEC-21-avg', 'DEC-21-std', 'Sprints-21', 'Sprints-21-avg',
       'Sprints-21-std'],
      dtype='object')

In [327]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 7449
Injury: 47
%: 0.63%


In [328]:
# Define columns to consider
columns_to_consider = [
    'TD-3', '>19.8-3', 'ACC-3', 'DEC-3', 'Sprints-3', 'TD-7', '>19.8-7',
    'ACC-7', 'DEC-7', 'Sprints-7', 'TD-21', '>19.8-21', 'ACC-21', 'DEC-21', 'Sprints-21'
]

# Count zero values in the specified columns for each row
zero_counts = (cumulative_df[columns_to_consider] == 0).sum(axis=1)

# Identify rows with 3 or more zeros
rows_with_3_or_more_zeros = cumulative_df[zero_counts >= 3]

# Display the first few rows where Injury == 1 and 3 or more zeros are present
print("Rows where Injury == 1 and 3 or more zeros:")
rows_with_3_or_more_zeros[rows_with_3_or_more_zeros['Injury'] == 1].head()

# Filter the DataFrame to exclude rows with 3 or more zeros
cumulative_df = cumulative_df[zero_counts < 3]


Rows where Injury == 1 and 3 or more zeros:


In [329]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Session,0
2,Injury,6832
3,season,0
4,preseason-season,0
...,...,...
63,DEC-21-avg,0
64,DEC-21-std,1
65,Sprints-21,0
66,Sprints-21-avg,0


In [330]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 6832
Injury: 44
%: 0.64%


In [331]:
cumulative_df[cumulative_df["Injury"]==1].sort_values(by="DATE", ascending=False).head(20)

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,TD-7-avg,TD-7-std,>19.8-7,>19.8-7-avg,>19.8-7-std,>25-7,>25-7-avg,>25-7-std,ACC-7,ACC-7-avg,ACC-7-std,DEC-7,DEC-7-avg,DEC-7-std,Sprints-7,Sprints-7-avg,Sprints-7-std,TD-21,TD-21-avg,TD-21-std,>19.8-21,>19.8-21-avg,>19.8-21-std,>25-21,>25-21-avg,>25-21-std,ACC-21,ACC-21-avg,ACC-21-std,DEC-21,DEC-21-avg,DEC-21-std,Sprints-21,Sprints-21-avg,Sprints-21-std
4958,2025-01-21,MD,1.0,NaN,NaN,2609.71,93.23,5.29,29.000000,27.0,98.0,1.000000,54.05,80.660000,NaN,NaN,2604.79,60.49,25.43,35.0,24.0,2.0,55.00,0.00,0.00,77.710000,9858.10,548.54,216.90,124.0,127.0,17.0,22716.64,3245.234286,3439.149911,1098.29,156.898571,234.255343,455.52,65.074286,103.766012,300.0,42.857143,43.233585,274.0,39.142857,46.340870,27.0,3.857143,6.121780,62579.206000,2979.962190,2942.497236,2763.400000,131.590476,185.405278,1180.55,56.216667,89.525954,751.700000,35.795238,35.211653,672.800000,32.038095,38.391802,61.000000,2.904762,4.459874
5181,2025-01-16,MD-2,1.0,NaN,NaN,411.78,0.00,0.00,0.000000,0.0,53.0,0.000000,16.00,34.830000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,12430.45,414.67,83.86,152.0,138.0,7.0,25027.10,3575.300000,3362.419767,704.82,100.688571,136.256679,189.83,27.118571,31.913443,273.0,39.000000,38.669540,240.0,34.285714,35.466953,14.0,2.000000,2.516611,76136.169048,3625.531859,3896.087201,3071.535556,146.263598,210.746036,1000.68,47.651429,83.474746,805.190476,38.342404,43.270689,697.111111,33.195767,39.258040,53.285714,2.537415,4.141356
7463,2025-01-09,MD,1.0,NaN,NaN,4351.15,189.35,24.06,59.000000,47.0,134.0,2.000000,60.00,82.770000,NaN,NaN,2268.61,34.60,0.00,34.0,32.0,0.0,39.00,0.00,0.00,70.590000,10428.79,484.59,25.19,146.0,118.0,3.0,16417.01,2345.287143,1970.188352,892.67,127.524286,181.323529,62.80,8.971429,15.735536,211.0,30.142857,25.504435,168.0,24.000000,23.057898,5.0,0.714286,1.253566,57490.500000,2737.642857,2435.139516,1899.410000,90.448095,144.694518,178.88,8.518095,19.061354,735.000000,35.000000,38.158878,607.000000,28.904762,39.007569,13.000000,0.619048,1.499206
6414,2024-12-23,MD-3,1.0,NaN,NaN,3994.16,462.50,33.30,74.000000,55.0,105.0,1.000000,60.00,92.570000,5.330000,9.860000,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,2816.16,138.33,26.75,32.0,25.0,3.0,15211.12,2173.017143,2450.622356,597.51,85.358571,117.397147,158.93,22.704286,35.766587,194.0,27.714286,33.609097,179.0,25.571429,33.195668,11.0,1.571429,2.299068,41009.890000,1952.851905,2206.297956,2000.440000,95.259048,126.222442,857.52,40.834286,79.530525,460.000000,21.904762,26.868020,470.000000,22.380952,26.971237,44.000000,2.095238,3.238900
7495,2024-10-19,MD,1.0,NaN,NaN,7690.38,280.94,56.40,79.000000,75.0,16.0,7.000000,96.04,96.130000,106.820000,6.600000,3499.06,55.40,118.60,45.0,32.0,2.0,70.00,32.90,98.60,96.810000,5267.59,55.40,118.60,56.0,39.0,2.0,5267.59,752.512857,1378.839490,55.40,7.914286,20.939232,118.60,16.942857,44.826586,56.0,8.000000,16.822604,39.0,5.571429,11.942322,2.0,0.285714,0.755929,38374.130000,1827.339524,3183.132602,903.800000,43.038095,96.199996,387.97,18.474762,39.770093,366.000000,17.428571,29.736462,350.000000,16.666667,30.643651,19.000000,0.904762,2.047065
7492,2024-10-05,MD,1.0,NaN,NaN,9989.17,275.93,110.63,93.000000,98.0,16.0,6.000000,122.26,96.730000,89.890000,66.600000,2510.23,9.26,0.00,22.0,24.0,0.0,57.00,0.00,0.00,77.050000,8822.71,226.72,85.81,94.0,84.0,6.0,23117.37,3302.481429,3680.351117,572.47,81.781429,127.777591,158.74,22.677143,38.505699,217.0,31.000000,33.020196,213.0,30.428571,35.250802,11.0,1.571429,2.699206,64887.480000,3089.880000,3043.696902,1764.160000,84.007619,121.117198,323.93,15.425238,29.628040,699.000000,33.285714,32.272500,578.000000,27.523810,30.986479,22.000000,1.047619,2.011870
6449,2024-09-21,MD,1.0,NaN,NaN,900.00,43.00,32.00,9.866667,10.8,48.0

## Calculate different load metrics

ACWR and MSWR values, trying to replicate the research paper

In [333]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6876 entries, 2 to 7640
Data columns (total 68 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              6876 non-null   datetime64[ns]
 1   Session           6693 non-null   object        
 2   Injury            6876 non-null   float64       
 3   season            4246 non-null   object        
 4   preseason-season  4246 non-null   object        
 5   TD                6876 non-null   float64       
 6   >19.8             6876 non-null   float64       
 7   >25               6876 non-null   float64       
 8   ACC               6876 non-null   float64       
 9   DEC               6876 non-null   float64       
 10  PlayerID          6876 non-null   float64       
 11  Sprints           6876 non-null   float64       
 12  Mins              6876 non-null   float64       
 13  % Max Speed       6873 non-null   float64       
 14  >19.8_Rel         6745 non-nu

In [334]:
min_threshold = 1e-5  # Small constant to replace zeros

cumulative_df['>19.8-7-std'] = cumulative_df['>19.8-7-std'].replace(0, min_threshold)
cumulative_df['>25-7-std'] = cumulative_df['>25-7-std'].replace(0, min_threshold)
cumulative_df['>19.8-21-avg'] = cumulative_df['>19.8-21-avg'].replace(0, min_threshold)
cumulative_df['>25-21-avg'] = cumulative_df['>25-21-avg'].replace(0, min_threshold)
cumulative_df['DEC-7-std'] = cumulative_df['DEC-7-std'].replace(0, min_threshold)
cumulative_df['ACC-7-std'] = cumulative_df['ACC-7-std'].replace(0, min_threshold)
cumulative_df['DEC-21-avg'] = cumulative_df['DEC-21-avg'].replace(0, min_threshold)
cumulative_df['ACC-21-avg'] = cumulative_df['ACC-21-avg'].replace(0, min_threshold)

In [335]:
def calculate_metrics_loads(df, metrics):

  # Calculate ACWR, MSWR for each metric
  for metric in metrics:

      # Calculate 7-day and 28-day averages for ACWR
      df[f'{metric}_ACWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-21-avg']

      # Calculate mean and standard deviation for MSWR
      df[f'{metric}_MSWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-7-std']

  return df

cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC']
cumulative_df = calculate_metrics_loads(cumulative_df, cols_calculate)

cumulative_df.drop(columns=['TD-7-avg', 'TD-7-std',
'>19.8-7-avg', '>19.8-7-std', '>25-7-avg',
       '>25-7-std', 'ACC-7-avg', 'ACC-7-std', 'DEC-7-avg',
       'DEC-7-std', 'Sprints-7-avg', 'Sprints-7-std','TD-21-avg', 'TD-21-std', '>19.8-21-avg', '>19.8-21-std', '>25-21-avg',
       '>25-21-std', 'ACC-21-avg', 'ACC-21-std',
       'DEC-21-avg', 'DEC-21-std', 'Sprints-21-avg',
       'Sprints-21-std'] , inplace=True)

In [336]:
cumulative_df.describe()

,DATE,Injury,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
count,6876,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6873.000000,6745.00000,6745.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000,6.876000e+03,6876.000000,6.876000e+03
mean,2023-06-12 19:56:13.821989376,0.006399,4739.066354,186.595425,42.777855,51.186070,43.259033,21463.602385,9.749600,68.092058,77.333779,28.09070,16.074293,2795.667712,96.009455,18.169104,32.673758,25.590431,4.882344,45.651568,14.442406,6.662019,53.719852,9425.149514,373.724560,81.711504,104.962329,86.389314,18.680857,21467.996736,847.673741,188.198312,236.509316,196.801051,42.443198,58396.389242,2317.492951,514.592734,638.235645,534.453991,114.430296,1.142583,1.289290,1.146054,0.806259,1.152065,0.569523,1.153780,1.789989e+03,1.153241,4.664684e+02
min,2021-08-12 00:00:00,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,4.040000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,492.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1378.000000,1.000000,0.000000,6.000000,2.000000,0.000000,2794.000000,2.000000,0.000000,15.000000,4.000000,1.000000,0.128481,0.377964,0.001156,0.377964,0.000000,0.000000,0.039848,3.779645e-01,0.014815,3.779645e-01
25%,2022-05-20 00:00:00,0.000000,2902.775000,37.000000,0.000000,30.000000,22.000000,134.000000,1.000000,53.000000,69.850000,0.51000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6662.935000,188.000000,14.000000,70.000000,55.000000,4.000000,17965.757500,576.662500,82.182500,183.000000,148.000000,11.000000,48246.674405,1653.000000,255.000000,484.000000,388.950000,31.000000,0.898079,0.883455,0.828413,0.614912,0.720488,0.453793,0.887555,8.705652e-01,0.880837,7.922616e-01
50%,2023-02-01 00:00:00,0.000000,4139.390000,116.000000,10.000000,47.000000,38.000000,17316.000000,5.000000,68.000000,78.908189,14.55000,0.530000,2924.000000,37.000000,0.000000,31.000000,22.000000,1.000000,54.820000,0.530000,0.000000,69.975523,9575.500000,353.620000,61.000000,102.000000,84.000000,12.000000,22135.000000,821.000000,161.940000,236.000000,197.000000,28.000000,62576.848000,2346.300000,447.000000,659.000000,552.000000,71.000000,1.025381,1.045374,1.019683,0.720310,1.033658,0.569951,1.026189,1.041448e+00,1.036007,9.716249e-01
75%,2024-10-05 00:00:00,0.000000,5769.000000,294.025000,62.000000,68.000000,58.000000,37709.000000,14.000000,83.600000,87.700000,44.09000,20.560000,4431.655000,133.000000,14.092500,53.000000,42.000000,7.000000,72.000000,18.650000,2.680000,80.406277,11870.250000,526.000000,124.000000,134.000000,113.000000,31.000000,25668.250000,1102.000000,268.000000,288.000000,244.000000,70.000000,71412.045000,2989.205000,731.090000,800.000000,678.000000,191.000000,1.215396,1.307673,1.301202,0.850083,1.426048,0.653901,1.252479,1.253628e+00,1.264266,1.170463e+00
max,2025-02-01 00:00:00,1.000000,24169.967520,1172.000000,438.460000,216.000000,206.000000,89871.000000,72.000000,253.683330,463.450000,327.00000,276.450000,13416.000000,1057.000000,366.840000,178.000000,146.000000,60.000000,133.000000,279.110000,242.320000,463.450000,30343.000000,1899.000000,499.000000,418.000000,336.000000,121.000

In [337]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6876 entries, 2 to 7640
Data columns (total 54 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              6876 non-null   datetime64[ns]
 1   Session           6693 non-null   object        
 2   Injury            6876 non-null   float64       
 3   season            4246 non-null   object        
 4   preseason-season  4246 non-null   object        
 5   TD                6876 non-null   float64       
 6   >19.8             6876 non-null   float64       
 7   >25               6876 non-null   float64       
 8   ACC               6876 non-null   float64       
 9   DEC               6876 non-null   float64       
 10  PlayerID          6876 non-null   float64       
 11  Sprints           6876 non-null   float64       
 12  Mins              6876 non-null   float64       
 13  % Max Speed       6873 non-null   float64       
 14  >19.8_Rel         6745 non-nu

# Do the shift for X previous days injury

If a player got injured X day, it means that the player has been loaded and could have happened the previous session or maybe in the previous 2 sessions.
To add more Injury columns, make the previous 2 days as Injury = 1.

In [338]:
cumulative_df

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
2,2022-10-31,MD+2,0.0,2022-2023,SEASON,4438.00,46.00,0.00,81.0,46.0,57090.0,7.0,59.90,73.833432,12.53,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,2417.00,12.00,1.00,18.0,18.0,1.0,6121.00,22.00,1.00,61.0,60.0,2.0,6121.00,22.00,1.00,61.0,60.0,2.0,1.000000,0.830104,1.000000,0.858956,1.000000,0.500000,1.000000,0.749270,1.000000,0.753778
3,2022-11-01,MD-1,0.0,2022-2023,SEASON,4023.00,151.00,0.00,29.0,17.0,57090.0,16.0,66.75,69.994093,41.12,0.00,4438.00,46.00,0.00,81.0,46.0,7.0,59.90,12.53,0.00,73.833432,4438.00,46.00,0.00,81.0,46.0,7.0,10559.00,68.00,1.00,142.0,106.0,9.0,10559.00,68.00,1.00,142.0,106.0,9.0,1.000000,1.025615,1.000000,0.717981,1.000000,0.447214,1.000000,0.828407,1.000000,0.958501
4,2022-11-02,MD,0.0,2022-2023,SEASON,3093.00,124.00,4.00,37.0,22.0,57090.0,14.0,31.42,77.554637,33.77,3.44,4023.00,151.00,0.00,29.0,17.0,16.0,66.75,41.12,0.00,69.994093,8461.00,197.00,0.00,110.0,63.0,23.0,14582.00,219.00,1.00,171.0,123.0,25.0,14582.00,219.00,1.00,171.0,123.0,25.0,1.000000,1.215083,1.000000,0.622909,1.000000,0.408248,1.000000,0.929419,1.000000,1.032382
5,2022-11-03,MD+1,0.0,2022-2023,SEASON,3908.00,27.00,0.00,47.0,39.0,57090.0,4.0,61.80,69.049025,7.35,0.00,3093.00,124.00,4.00,37.0,22.0,14.0,31.42,33.77,3.44,77.554637,11554.00,321.00,4.00,147.0,85.0,37.0,17675.00,343.00,5.00,208.0,145.0,39.0,17675.00,343.00,5.00,208.0,145.0,39.0,1.000000,1.370074,1.000000,0.779153,1.000000,0.477455,1.000000,1.054584,1.000000,1.142181
6,2022-11-04,MD-1,0.0,2022-2023,SEASON,4014.00,138.00,0.00,13.0,13.0,57090.0,7.0,70.48,70.998228,37.58,0.00,3908.00,27.00,0.00,47.0,39.0,4.0,61.80,7.35,0.00,69.049025,11024.00,302.00,4.00,113.0,78.0,34.0,17879.00,360.00,5.00,212.0,142.0,42.0,21583.00,370.00,5.00,255.0,184.0,43.0,0.946724,1.368645,1.111969,0.837008,1.142857,0.477455,0.950140,1.061576,0.881988,1.154192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7636,2025-01-28,MD-4,0.0,NaN,NaN,5175.50,131.56,3.29,73.0,67.0,177.0,0.0,79.00,80.440000,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,3845.46,176.63,42.13,33.0,25.0,3.0,17578.75,723.11,123.46,228.0,232.0,9.0,42059.07,1632.81,324.02,599.0,580.0,25.0,0.776200,0.852903,0.822459,0.792901,0.707619,0.798871,0.706892,0.816170,0.742857,0.748339
7637,2025-01-29,MD-3,0.0,NaN,NaN,4241.49,322.83,166.74,57.0,55.0,177.0,8.0,62.00,102.630000,0.00,0.00,5175.50,131.56,3.29,73.0,67.0,0.0,79.00,0.00,0.00,80.440000,5175.50,131.56,3.29,73.0,67.0,0.0,14789.68,514.66,82.94,191.0,175.0,5.0,47234.57,1764.37,327.31,672.0,647.0,25.0,0.626223,0.973470,0.583392,0.895977,0.506798,0.617260,0.568452,0.945259,0.540958,0.945361
7638,2025-01-30,MD-2,0.0,NaN,NaN,1570.44,5.96,0.00,3.0,10.0,177.0,0.0,34.00,62.750000,0.00,0.00,4241.49,322.83,166.74,57.0,55.0,8.0,62.00,0.00,0.00,102.630000,9416.99,454.39,170.03,130.0,122.0,8.0,19031.17,837.49,249.68,248.0,230.0,13.0,51476.06,2087.20,494.05,729.0,702.0,33.0,0.792234,1.312088,0.859822,1.021781,1.082944,0.587846,0.728983,1.269399,0.702076,1.266415
7639,2025-01-31,MD-1,0.0,NaN,NaN,2673.04,30.45,0.00,53.0,48.0,177.0,0.0,62.00,70.020000,0.00,0.00,1570.44,5.96,0.00,3.0,10.0,0.0,34.00,0.00,0.00,62.750000,10987.43,460.35,170.03,133.0,132.0,8.0,16911.34,675.33,212.16,199.0,196.0,11.0,53046.50,2093.16,494.05,732.0,712.0,33.0,0.728691,1.172033,0.737455,0.791784,0.981555,0.487858,0.621390,0.974355,0.629213,1.049344


In [339]:
def shift_injuries_x_days(df, x):
    copy_df = df.copy()

    # Iterate over the df to shift injury labels
    for idx, row in copy_df.iterrows():
        if row['Injury'] == 1:
            # Get the player's ID and the injury date
            player_id = row['PlayerID']
            injury_date = row['DATE']

            # Shift injury labels for the current day and the previous x days
            for i in range(0, x + 1):
                prev_date = pd.to_datetime(injury_date) - pd.Timedelta(days=i)
                mask = (copy_df['PlayerID'] == player_id) & (copy_df['DATE'] == prev_date)
                copy_df.loc[mask, 'Injury'] = 1

    return copy_df

In [340]:
# One previous day
# df_injury_shifted = shift_injuries_x_days(cumulative_df, 1)
df_injury_shifted = cumulative_df.copy()

In [341]:
print("No Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 0].shape[0]))
print("Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 1].shape[0]))
print(f"%: {((df_injury_shifted[df_injury_shifted['Injury'] == 1].shape[0] / df_injury_shifted.shape[0]) * 100):.2f}%")

No Injury: 6832
Injury: 44
%: 0.64%


In [342]:
df_injury_shifted.head()

,DATE,Session,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR
2,2022-10-31,MD+2,0.0,2022-2023,SEASON,4438.0,46.0,0.0,81.0,46.0,57090.0,7.0,59.90,73.833432,12.53,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,2417.0,12.0,1.0,18.0,18.0,1.0,6121.0,22.0,1.0,61.0,60.0,2.0,6121.0,22.0,1.0,61.0,60.0,2.0,1.000000,0.830104,1.000000,0.858956,1.000000,0.500000,1.00000,0.749270,1.000000,0.753778
3,2022-11-01,MD-1,0.0,2022-2023,SEASON,4023.0,151.0,0.0,29.0,17.0,57090.0,16.0,66.75,69.994093,41.12,0.00,4438.0,46.0,0.0,81.0,46.0,7.0,59.90,12.53,0.00,73.833432,4438.0,46.0,0.0,81.0,46.0,7.0,10559.0,68.0,1.0,142.0,106.0,9.0,10559.0,68.0,1.0,142.0,106.0,9.0,1.000000,1.025615,1.000000,0.717981,1.000000,0.447214,1.00000,0.828407,1.000000,0.958501
4,2022-11-02,MD,0.0,2022-2023,SEASON,3093.0,124.0,4.0,37.0,22.0,57090.0,14.0,31.42,77.554637,33.77,3.44,4023.0,151.0,0.0,29.0,17.0,16.0,66.75,41.12,0.00,69.994093,8461.0,197.0,0.0,110.0,63.0,23.0,14582.0,219.0,1.0,171.0,123.0,25.0,14582.0,219.0,1.0,171.0,123.0,25.0,1.000000,1.215083,1.000000,0.622909,1.000000,0.408248,1.00000,0.929419,1.000000,1.032382
5,2022-11-03,MD+1,0.0,2022-2023,SEASON,3908.0,27.0,0.0,47.0,39.0,57090.0,4.0,61.80,69.049025,7.35,0.00,3093.0,124.0,4.0,37.0,22.0,14.0,31.42,33.77,3.44,77.554637,11554.0,321.0,4.0,147.0,85.0,37.0,17675.0,343.0,5.0,208.0,145.0,39.0,17675.0,343.0,5.0,208.0,145.0,39.0,1.000000,1.370074,1.000000,0.779153,1.000000,0.477455,1.00000,1.054584,1.000000,1.142181
6,2022-11-04,MD-1,0.0,2022-2023,SEASON,4014.0,138.0,0.0,13.0,13.0,57090.0,7.0,70.48,70.998228,37.58,0.00,3908.0,27.0,0.0,47.0,39.0,4.0,61.80,7.35,0.00,69.049025,11024.0,302.0,4.0,113.0,78.0,34.0,17879.0,360.0,5.0,212.0,142.0,42.0,21583.0,370.0,5.0,255.0,184.0,43.0,0.946724,1.368645,1.111969,0.837008,1.142857,0.477455,0.95014,1.061576,0.881988,1.154192


# One Hot Encoding

One Hot Encoding for Match Day

In [343]:
# Step 2: Sort by PlayerID and Date
df_injury_shifted.sort_values(by=['PlayerID', 'DATE'], inplace=True)

# Step 3: Shift the Session column by -1 for each PlayerID
df_injury_shifted['Session'] = df_injury_shifted.groupby('PlayerID')['Session'].shift(1)

# Step 2: Perform one-hot encoding
final_encoded_final_df = pd.get_dummies(df_injury_shifted, columns=['Session'], prefix='Session')

one_hot_columns = [col for col in final_encoded_final_df.columns if col.startswith('Session_')]
final_encoded_final_df[one_hot_columns] = final_encoded_final_df[one_hot_columns].astype(int)

final_encoded_final_df = final_encoded_final_df.dropna(subset=one_hot_columns)
# Final encoded DataFrame
print(final_encoded_final_df)

           DATE  Injury     season preseason-season        TD   >19.8     >25  \
7547 2024-11-28     0.0        NaN              NaN   5673.60  227.63  106.23   
7548 2024-11-29     0.0        NaN              NaN   3187.82   29.50    0.00   
7549 2024-11-30     0.0        NaN              NaN  11860.70  525.95  239.76   
7550 2024-12-02     0.0        NaN              NaN   3881.53  150.27    0.00   
7551 2024-12-03     0.0        NaN              NaN   3256.74   18.06    0.00   
...         ...     ...        ...              ...       ...     ...     ...   
3496 2023-04-21     0.0  2022-2023           SEASON   4192.00  155.00    0.00   
3497 2023-04-22     0.0  2022-2023           SEASON   4572.00  214.00  101.00   
3498 2023-04-24     0.0  2022-2023           SEASON   3788.00  136.00   12.00   
3499 2023-04-25     0.0  2022-2023           SEASON   4871.00   83.00    0.00   
3501 2023-05-06     0.0  2022-2023           SEASON   2981.00   66.00    0.00   

        ACC    DEC  PlayerI

In [344]:
final_encoded_final_df.head()

,DATE,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Sprints,Mins,% Max Speed,>19.8_Rel,>25_Rel,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,>19.8_Rel-1,>25_Rel-1,% Max Speed-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,TD_ACWR,TD_MSWR,>19.8_ACWR,>19.8_MSWR,>25_ACWR,>25_MSWR,ACC_ACWR,ACC_MSWR,DEC_ACWR,DEC_MSWR,Session_MD,Session_MD+1,Session_MD+2,Session_MD+3,Session_MD-1,Session_MD-2,Session_MD-3,Session_MD-4,Session_MD-5
7547,2024-11-28,0.0,NaN,NaN,5673.60,227.63,106.23,48.0,45.0,4.0,6.0,84.72,100.85,40.60,15.35,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,9643.96,568.92,70.39,116.0,115.0,8.0,12066.12,616.88,70.39,146.0,142.0,8.0,16919.31,832.51,103.42,208.0,193.0,11.0,2.139470,0.477850,2.222964,0.414198,2.041868,0.377964,2.105769,0.480378,2.207254,0.472199,0,0,0,0,0,0,0,0,0
7548,2024-11-29,0.0,NaN,NaN,3187.82,29.50,0.00,44.0,38.0,4.0,0.0,56.67,72.32,0.00,0.00,5673.60,227.63,106.23,48.0,45.0,6.0,84.72,40.6,15.35,100.85,5673.60,227.63,106.23,48.0,45.0,6.0,15317.56,796.55,176.62,164.0,160.0,14.0,22592.91,1060.14,209.65,256.0,238.0,17.0,2.033943,0.559804,2.254089,0.522261,2.527355,0.569359,1.921875,0.525684,2.016807,0.519999,0,0,0,0,0,1,0,0,0
7549,2024-11-30,0.0,NaN,NaN,11860.70,525.95,239.76,125.0,106.0,4.0,16.0,128.30,97.02,90.74,43.74,3187.82,29.50,0.00,44.0,38.0,0.0,56.67,0.0,0.00,72.32,8861.42,257.13,106.23,92.0,83.0,6.0,18505.38,826.05,176.62,208.0,198.0,14.0,25780.73,1089.64,209.65,300.0,276.0,17.0,2.153397,0.696509,2.274283,0.547367,2.527355,0.569359,2.080000,0.678267,2.152174,0.657824,0,0,0,0,1,0,0,0,0
7550,2024-12-02,0.0,NaN,NaN,3881.53,150.27,0.00,54.0,34.0,4.0,0.0,52.42,76.35,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,15048.52,555.45,239.76,169.0,144.0,16.0,30366.08,1352.00,416.38,333.0,304.0,30.0,37641.43,1615.59,449.41,425.0,382.0,33.0,2.420159,0.884554,2.510538,0.755778,2.779511,0.658507,2.350588,0.881363,2.387435,0.876564,1,0,0,0,0,0,0,0,0
7551,2024-12-03,0.0,NaN,NaN,3256.74,18.06,0.00,20.0,14.0,4.0,0.0,74.30,69.70,0.00,0.00,3881.53,150.27,0.00,54.0,34.0,0.0,52.42,0.0,0.00,76.35,15742.23,676.22,239.76,179.0,140.0,16.0,24603.65,933.35,345.99,271.0,223.0,22.0,41522.96,1765.86,449.41,479.0,416.0,33.0,1.777594,0.814913,1.585658,0.684837,2.309628,0.532634,1.697286,0.855423,1.608173,0.833889,0,0,0,0,0,1,0,0,0


# Export data

In [64]:
#encoded_final_df.to_excel('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy2.xlsx', index=False)

In [1]:
final_encoded_final_df.to_excel('data/new_data_no_injured.xlsx', index=False)

NameError: name 'final_encoded_final_df' is not defined

In [229]:
final_encoded_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6896 entries, 7547 to 3501
Data columns (total 62 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              6896 non-null   datetime64[ns]
 1   Injury            6896 non-null   float64       
 2   season            4246 non-null   object        
 3   preseason-season  4246 non-null   object        
 4   TD                6896 non-null   float64       
 5   >19.8             6896 non-null   float64       
 6   >25               6896 non-null   float64       
 7   ACC               6896 non-null   float64       
 8   DEC               6896 non-null   float64       
 9   PlayerID          6896 non-null   float64       
 10  Sprints           6896 non-null   float64       
 11  Mins              6896 non-null   float64       
 12  % Max Speed       6896 non-null   float64       
 13  >19.8_Rel         6768 non-null   float64       
 14  >25_Rel           6768 non

In [ ]:
final_encoded_final_df[(final_encoded_final_df['19.8_Rel-1'].isna()) | (final_encoded_final_df['25_Rel-1'].isna())]
